<a href="https://colab.research.google.com/github/AkRypt/open_source_rag_learning/blob/main/LlamaRAGBasic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pypdf accelerate langchain bitsandbytes sentence_transformers

In [ ]:
!pip install llama-index
!pip install llama-index-llms-huggingface
!pip install llama-index-embeddings-langchain

In [3]:
from llama_index.core import set_global_service_context, ServiceContext
from llama_index.core import VectorStoreIndex, download_loader, SimpleDirectoryReader
from llama_index.core.prompts.prompts import SimpleInputPrompt
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.langchain import LangchainEmbedding
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

In [6]:
system_prompt="""
You are a helpful assistant. Your goal is to answer a user's
questions based on the instructions and context provided.
"""
## Default format supportable by LLama2
query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [22]:
from google.colab import userdata
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!huggingface-cli login --token userdata.get('hf_token')

/bin/bash: -c: line 1: syntax error near unexpected token `('
/bin/bash: -c: line 1: `huggingface-cli login --token userdata.get('hf_token')'


In [11]:
auth_token = ""
model_name = "meta-llama/Llama-2-7b-chat-hf"

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name=model_name,
    model_name=model_name,
    device_map="auto",
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [12]:
embed_model = LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

<ipython-input-13-6394536e40c5>:1: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [23]:
documents = SimpleDirectoryReader("/content/data").load_data()
index = VectorStoreIndex.from_documents(documents,service_context=service_context)
query_engine = index.as_query_engine()

In [18]:
response = query_engine.query("What kind of a job can I hire Akshit for?")
response.response

"\nBased on the provided context information, Akshit's skills and experience, he can be hired for a full-stack developer or software engineer role. His expertise in programming languages such as Python, Java, JavaScript, TypeScript, C++, C, C#, React, Angular, SQL, Node, Flutter, Riverpod, React Native, Next.js, Supabase, Git, HTML, CSS, Tailwind CSS, PostgreSQL, UI/UX, Android, NestJS, Jira, Ruby, NoSQL, jQuery, GraphQL, AWS, GCP, Azure, CI/CD, Jest, Unit Testing, OOP, RESTful API, data structures, algorithms, iOS, Redux, Express, Vue, Linux, MongoDB, microservices architecture, cloud computing, frontend, backend, full-stack, Confluence, Kafka, Ansible, etc. makes him a suitable candidate for these roles. Additionally, his experience in leading development teams, implementing personalized customer experiences, and optimizing user interactions with data can be valuable in a software development or engineering role."

In [26]:
# @title Enter your query and run:
user_query = "Where did Akshit give his GRE test?" # @param {type:"string"}
response = query_engine.query(user_query)
response.response

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


'According to the provided score report, Akshit gave his GRE test on February 4, 2021, at Hyderabad, IN-TG, 500004 India.'